## (1)ライブラリのインポート

In [14]:
import torch 
import torch.nn as nn 
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torchvision.datasets import ImageFolder # 【追加】 画像データセットを提供するモジュール
from torchvision import models # 【追加】 事前学習済みモデルや一般的なモデルアーキテクチャを含むモジュール

## (2)データの前処理

In [15]:
# (2-D) データの前処理
data_transform = {
    'train': transforms.Compose([ 
        transforms.Resize((224, 224)), 
        transforms.RandomHorizontalFlip(), 
        transforms.RandomRotation(10), 
        transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNetデータセットの平均と標準偏差を使用

    ]),
    'val': transforms.Compose([ 
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNetデータセットの平均と標準偏差を使用、検証データに対しても同様に正規化を行う。理由は、学習時と同じ前処理を行うことで、モデルの性能を正確に評価するため。
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNetの平均と標準偏差を使用
    ]),
}

## (3)データセットの用意

In [ ]:
full_dataset = ImageFolder(root='Dataset-name')

# データセットを70%訓練、15%検証、15%テストに分割
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size]
)

# 各データセットに適切な変換を適用するためのラッパークラスを定義
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform):
        self.subset = subset  # Subsetオブジェクト
        self.transform = transform  # 適用する変換

    def __getitem__(self, index):
        image, label = self.subset[index]  # 画像とラベルを取得
        if self.transform:
            image = self.transform(image)  # 変換を適用
        return image, label

    def __len__(self):
        return len(self.subset)
    
# それぞれのデータセットに対応する変換を適用（ここで初めてデータを前処理する）
train_dataset = CustomDataset(train_dataset, transform=data_transform['train'])
val_dataset = CustomDataset(val_dataset, transform=data_transform['val'])
test_dataset = CustomDataset(test_dataset, transform=data_transform['val'])

## (4) データローダーの作成

In [17]:
BATCH_SIZE = 64  # ミニバッチサイズを設定、データセットのサイズによって自分で設定する
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


## (5) ハードウェアの設定

In [18]:
# ハードウェアの設定 
device = torch.device('mps')

## (6) 入力データのテンソル形状の確認

In [ ]:
# ニューラルネットワークに入力する画像のサイズ、チャネル数をチェック
images, labels = next(iter(train_loader))
c, h, w = images[0].shape
print("ミニバッチサイズ: ", images.size())
print("画像のチャネル数: ", c)
print("画像の高さ: ", h)
print("画像の幅: ", w)

# グリッド乗に4枚の画像を表示
img = torchvision.utils.make_grid(images[:4])
img = transforms.functional.to_pil_image(img)
display(img)

# 上の画像に対応するラベルを表示
print(labels[:4])


## (7) ネットワーク構成の定義

In [ ]:
# ニューラルネットワークに入力する画像のサイズと分類するクラス数を定義
INPUT_CHANNELS = c
OUTPUT_SIZE = 10  # 分類するクラス数

model = torchvision.models.mobilenet_v2(weights="IMAGENET1K_V1")

# 最終層の出力ユニット数を変更
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, OUTPUT_SIZE)

# モデルをデバイスに転送
model = model.to(device)
model

## (8) 損失関数の定義

In [21]:
# 多クラス分類のための損失関数を定義
criterion = nn.CrossEntropyLoss()

## (9) 最適化関数の定義

In [22]:
# Adam, weight_decayで化学集対策
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

## (10) モデル学習＆検証

In [ ]:
from tqdm import tqdm  # tqdmをインポート

# モデルの学習
EPOCHS = 10
train_loss_list = []  # 訓練データの損失リスト
val_loss_list = []    # 検証データの損失リスト
train_acc_list = []   # 訓練データの精度リスト
val_acc_list = []     # 検証データの精度リスト

for epoch in range(EPOCHS):
    model.train()  # モデルを学習モードに設定
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    # 訓練データのループにtqdmを追加
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [訓練]', leave=False):
        images, labels = images.to(device), labels.to(device)  # データをデバイスに転送
        optimizer.zero_grad()  # 勾配を初期化

        outputs = model(images)  # モデルの出力を取得
        loss = criterion(outputs, labels)  # 損失を計算

        loss.backward()  # 逆伝播
        optimizer.step()  # パラメータを更新

        train_loss += loss.item() * images.size(0)  # 損失を蓄積（バッチサイズで重み付け）

        # 正解数を計算
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_loss /= train_total  # データ数で割って平均を取る
    train_accuracy = 100 * train_correct / train_total  # 訓練データの精度計算
    train_loss_list.append(train_loss)  # 訓練損失をリストに追加
    train_acc_list.append(train_accuracy)  # 訓練精度をリストに追加

    model.eval()  # モデルを評価モードに設定
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        # 検証データのループにtqdmを追加
        for images, labels in tqdm(val_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [検証]', leave=False):
            images, labels = images.to(device), labels.to(device)  # データをデバイスに転送

            outputs = model(images)  # モデルの出力を取得
            loss = criterion(outputs, labels)  # 損失を計算

            val_loss += loss.item() * images.size(0)  # 損失を蓄積（バッチサイズで重み付け）

            # 正解数を計算
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= val_total  # データ数で割って平均を取る
    val_accuracy = 100 * val_correct / val_total  # 検証データの精度計算
    val_loss_list.append(val_loss)  # 検証損失をリストに追加
    val_acc_list.append(val_accuracy)  # 検証精度をリストに追加

    # エポックごとの結果を表示
    print(f'Epoch {epoch+1}/{EPOCHS}, '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

## (11) モデル学習結果表示

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# 検証データでの推論
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in val_loader:  # val_loader: 検証データのDataLoader
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# クラス名とラベルを一致させる
unique_labels = sorted(set(all_labels))  # 出現するクラスラベルを取得
if 'class_names' not in globals() or len(class_names) != len(unique_labels):
    class_names = [f"Class {i}" for i in unique_labels]  # 自動生成（例: "Class 0", "Class 1", ...）

# 混同行列の作成
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

# 混同行列の可視化
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Precision, Recall, F1スコアのレポート
report = classification_report(all_labels, all_preds, target_names=class_names)
print("Classification Report:")
print(report)


## (12) テストデータを用いてモデルの性能を評価し、正解率を算出

In [ ]:
# モデルの評価
model.eval() # モデルを評価モードに設定
correct = 0 # 正解数
total = 0 # テストデータの総数
with torch.no_grad():
    for images, labels in test_loader: # テストデータを取得
        images, labels = images.to(device), labels.to(device) # データをハードウェアに転送
        outputs = model(images) # 画像をモデルに入力して出力を取得
        _, predicted = torch.max(outputs, 1) # 確率が最大のラベルを取得=モデルによる予測結果を取得
        total += labels.size(0) # labels.size(0)はミニバッチサイズ、毎回加算することでテストデータの総数を計算
        correct += (predicted == labels).sum().item() # 予測と正解ラベルが一致した場合に正解数をカウント
# 正解率を%で表示
print(f'Accuracy: {correct / total * 100:.2f}%')

## (13) モデルパラメータのファイル保存

In [ ]:
# 学習済みの重みを保存
# model.state_dict(): モデルの重みを格納したオブジェクト
torch.save(model.state_dict(), "model.pth")